## Multi-Process One-Thread

In [1]:
import multiprocessing
from multiprocessing import Process

multiprocessing chạy paralell kết hợp concurency

In [3]:
# Check number of available cores
print("Number of cpu cores: ", multiprocessing.cpu_count())

Number of cpu cores:  20


In [4]:
import pandas as pd
from time import sleep, time

# Example of a processing function
def process_dataframe(chunk_id, chunk_data: pd.DataFrame):
    print(f"Processing chunk {chunk_id}")
    sleep(5)
    print(f"The chunk {chunk_id} has been processed successfully!")

In [5]:
# Make a sample dataframe
data = [
    ['tom', 10], ['nick', 15],
    ['juli', 14], ['peter', 20],
    ['jason', 27], ['anna', 11]
]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['Name', 'Age'])

# Devide the dataframe into chunks
chunk_size = 2
chunks = [df[i:i+chunk_size].to_numpy() for i in range(0, len(df), chunk_size)]

# Mark the starting point to measure processing time
start = time()

procs = []
#lặp qua các chunk, mỗi chunk đưa vào 1 proc
# không đồng đều do đôi khi corelogic đang bận xử lý cái khác chưa xử lý kịp hàm process_dataframe
# tuy nhiên Mỗi lần Process(...) được tạo và start(), hệ điều hành sẽ gán tiến trình đó vào một logic core cụ thể 
# (hoặc luân chuyển giữa các core nếu cần), nhưng việc gán vào core nào là do hệ điều hành quyết định.
for i, chunk in enumerate(chunks):
  # Define our process but not yet start
  proc = Process(target=process_dataframe, args=(i, chunk,))
  # Start the process
  proc.start()
  # Investigate process ID
  print(f"Process ID: {proc.pid}")
  # Manage all process definitions in a list
  procs.append(proc)

# Stop all processes to prevent resource scarcity
# imagine zombie processes
# chờ các proc xử lý xong hết để tránh tình trạng zombie proc
for proc in procs:
    proc.join() # Wait for the process to finish

# Report total elapsed time
# nếu không có join phía trên đôi khi dòng dưới còn chạy xong trước
print(f"Total time: {time() - start}s")

Process ID: 24243
Processing chunk 0
Process ID: 24248


Processing chunk 1
Processing chunk 2
Process ID: 24251
The chunk 0 has been processed successfully!
The chunk 1 has been processed successfully!
The chunk 2 has been processed successfully!
Total time: 5.08407735824585s


In [7]:
##############################################
# The 2nd way to do multiprocess is via Pool #
##############################################
from multiprocessing import Pool

# Define a pool of processes
NUM_PROCESSES = 2 # có 2 proc đc tạo ra
pool = Pool(NUM_PROCESSES)

procs = []
for i, chunk in enumerate(chunks):
    procs.append(
        # The main process does not need to wait for this function
        pool.apply_async(process_dataframe, args=(i, chunk))
    )

for proc in procs:
    proc.get() # Wait for the process to finish

Processing chunk 0Processing chunk 1



The chunk 1 has been processed successfully!The chunk 0 has been processed successfully!

Processing chunk 2
The chunk 2 has been processed successfully!


In [6]:
##############################################
# The 3rd way to do multiprocess is via Map #
##############################################
inputs = [(i, chunk) for i, chunk in enumerate(chunks)]

# If you have one arg only, please use `.map` instead
outputs = pool.starmap(
    process_dataframe,
    inputs
)

In [7]:
# Uhm... you can see that sometimes, two strings are concatnated w/o any `/n` ( race condition problem)
# this is because all the processes writes to stdout at the same time, we need
# to find a way to prevent other processes write to it while one is doing
from multiprocessing import Lock

lock = Lock()

def process_dataframe(chunk_id, chunk_data: pd.DataFrame):
    lock.acquire() #process đang xét đang có quyền cầm khóa
    # do đang cầm khóa nên đây là thằng duy nhất được ghi vào terminal
    #Mỗi khi bạn dùng lock.acquire(), bạn đang nói với chương trình rằng “đoạn mã tiếp theo cần được thực hiện theo 
    # cách an toàn, chỉ có một luồng duy nhất được phép thao tác vào tài nguyên này tại một thời điểm”.
    print(f"Processing chunk {chunk_id}")
    sleep(5)
    print(f"The chunk {chunk_id} has been processed successfully!")
    lock.release() #process đang xét nhả khóa ra

# Make a sample dataframe
data = [
    ['tom', 10], ['nick', 15],
    ['juli', 14], ['peter', 20],
    ['jason', 27], ['anna', 11]
]

# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['Name', 'Age'])

# Devide the dataframe into chunks
chunk_size = 2
chunks = [df[i:i+chunk_size] for i in range(0, len(df), chunk_size)]

# Mark the starting point to measure processing time
start = time()

procs = []
for i, chunk in enumerate(chunks):
  # Define our process but not yet start
  proc = Process(target=process_dataframe, args=(i, chunk,))
  # Start the process
  proc.start()
  # Manage all process definitions in a list
  procs.append(proc)

# Stop all processes to prevent resource scarcity
# imagine zombie processes
for proc in procs:
    proc.join()

# Report total elapsed time
print(f"Total time: {time() - start}s")

Processing chunk 0
The chunk 0 has been processed successfully!
Processing chunk 1
The chunk 1 has been processed successfully!
Processing chunk 2
The chunk 2 has been processed successfully!
Total time: 15.079243898391724s


## Multi-Threaded

Python provides the same APIs to multiprocessing with `start()` and `join()`.

Thread cùng chia sẻ vùng nhớ với nhau khi nó cùng trong process ( proc có thể tạo nhiều thread) 
GIL ( Khi có 1 proc có khả năng excute nhiều task, mỗi task sẽ được xử lý bởi các thread -> xảy ra tình trạng thread xử lý task này nhảy sang task khác -> cùng cập nhật variable X)

🧠 GIL (Global Interpreter Lock) là gì?

GIL là một cơ chế trong CPython (trình thông dịch Python phổ biến nhất) đảm bảo rằng chỉ một thread Python được thực thi tại một thời điểm, ngay cả khi bạn sử dụng nhiều thread.

🤔 Tại sao lại có GIL?

CPython không thread-safe khi truy cập vào các đối tượng Python trong bộ nhớ.

GIL giúp đơn giản hóa việc quản lý bộ nhớ và garbage collection.

Nhưng đổi lại, nó gây ra hạn chế với các chương trình đa luồng (multi-threading), đặc biệt khi xử lý tác vụ CPU-bound.

📌 Khi nào GIL không phải là vấn đề?

Với I/O-bound tasks (như đọc ghi file, gọi API, chờ mạng...), GIL không ảnh hưởng nhiều.

Với CPU-bound tasks (tính toán nặng), GIL làm multi-threading kém hiệu quả. Lúc này, nên dùng multiprocessing (đa tiến trình) thay vì đa luồng.

📦 Một số cách “vượt qua” GIL:

Sử dụng multiprocessing thay vì threading

Dùng thư viện C mở rộng (như NumPy, hoặc các thư viện Cython cho phép thả GIL)

Chuyển sang trình thông dịch không có GIL, như:

PyPy (dù hiện vẫn có GIL)

Jython (chạy trên JVM, không có GIL)

IronPython (chạy trên .NET CLR)

In [8]:
from threading import Thread

my_threads = []
for i, chunk in enumerate(chunks):
    my_thread = Thread(target=process_dataframe, args=(i, chunk,))
    my_thread.start()
    my_threads.append(my_thread)

for my_thread in my_threads:
    my_thread.join()

Processing chunk 0
The chunk 0 has been processed successfully!
Processing chunk 1
The chunk 1 has been processed successfully!
Processing chunk 2
The chunk 2 has been processed successfully!


However, the way they share data is different. Let's take a look at the example below

In [13]:
from threading import Lock

# Define a lock to prevent race condition,
# which is multiple updates into the same variable
lock = Lock()

# Share data
shared_data = 0

def increment_function():
    global shared_data
    with lock: # This is equal to acquire() + release()
        shared_data += 1

my_threads = []
for _ in range(3):
    my_thread = Thread(target=increment_function)
    my_thread.start()
    my_threads.append(my_thread)

for my_thread in my_threads:
    my_thread.join()

print(f"Current value of shared_data: {shared_data}")

Current value of shared_data: 3


In [14]:
# Another way to access shared_data is via Queue
from queue import Queue

# Initialize the queue
shared_queue = Queue()

def increment_function():
    shared_queue.put(1)

my_threads = []
for _ in range(3):
    my_thread = Thread(target=increment_function)
    my_thread.start()
    my_threads.append(my_thread)

for i, my_thread in enumerate(my_threads):
    my_thread.join()

shared_data = 0
while not shared_queue.empty():
    shared_data += shared_queue.get()

print(f"Current value of shared_data: {shared_data}")

Current value of shared_data: 3


In [ ]:
# Using a global variable is not a piece of cake in multiprocessing
# as in multithreading
import multiprocessing

# Define a shared value between oprocesses
shared_variable = multiprocessing.Value('i', 0)

def worker_function():
    global shared_variable
    with shared_variable.get_lock():
        shared_variable.value += 1

procs = []
for _ in range(3):
  # Define our process but not yet start
  proc = Process(target=worker_function)
  # Start the process
  proc.start()
  # Manage all process definitions in a list
  procs.append(proc)

for proc in procs:
    proc.join()

print(f"Current value of shared_data: {shared_data}")

In [ ]:
# Or using queue
from multiprocessing import Queue
from multiprocessing import Process

def worker_function(shared_queue):
    shared_queue.put(1)

shared_queue = Queue()

procs = []
for _ in range(3):
    my_proc = Process(target=worker_function, args=(shared_queue,))
    my_proc.start()
    procs.append(my_proc)

for proc in procs:
    proc.join()

# The main process retrieves data from the queue
# and aggregate the result
result = 0
while not shared_queue.empty():
    result += shared_queue.get(1)

print(f"Results from multiprocessing queue: {result}")

## One-Process One-Thread
### Vào event_loop_demo để test kỹ

In [15]:
# Let's say you have one process with one thread only, how to deal with it?
# AsyncIO helps us to do it
#NOTE: CHỈ NÊN DÙNG AWAIT CHO CÁC TÁC VỤ I/O
import asyncio

#coroutine
async def download_my_1st_data_func():
    print("Starting my 1st data func...")
    await asyncio.sleep(2)
    print("Completed my 1st data func!")

async def download_my_2nd_data_func():
    print("Starting my 2nd data func...")
    await asyncio.sleep(2)
    print("Completed my 2nd data func!")

def main():
    loop = asyncio.get_event_loop() # nơi chứa task 1 thread cần thực hiện
    loop.run_until_complete(asyncio.gather(
        download_my_1st_data_func(),
        download_my_2nd_data_func(),
    ))
    loop.close()

# Run the event loop
main()

# YOU WILL MEET SOME WEIRD ERRORS HERE DUE TO IPYTHON NOTEBOOK,
# LET'S MOVE TO A PYTHON SCRIPT

RuntimeError: This event loop is already running

Starting my 1st data func...
Starting my 2nd data func...
Completed my 1st data func!
Completed my 2nd data func!
